In [1]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [2]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("Spoon")

LangSmith 추적을 시작합니다.
[프로젝트명]
Spoon


In [7]:
import os
import requests
import zipfile
import io
from typing import List, Dict, Any
from lxml import etree
from langchain_community.document_loaders import BSHTMLLoader
import tempfile
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.chains.llm import LLMChain
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
import pandas as pd
import logging

# 로깅 설정
logging.basicConfig(
    level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s"
)

# 상수 정의
API_URL = "https://opendart.fss.or.kr/api/document.xml"
CHUNK_SIZE = 4000
CHUNK_OVERLAP = 0
LLM_MODEL = "gpt-4o-mini-2024-07-18"
LLM_TEMPERATURE = 0


def fetch_document(api_key: str, rcp_no: str) -> bytes:
    """DART API를 통해 문서를 가져옵니다."""
    params = {"crtfc_key": api_key, "rcept_no": rcp_no}
    response = requests.get(API_URL, params=params)
    response.raise_for_status()
    return response.content


def extract_section(
    root: etree.Element, start_aassocnote: str, end_aassocnote: str
) -> str:
    """XML에서 특정 섹션을 추출합니다."""
    start_element = root.xpath(
        f"//TITLE[@ATOC='Y' and @AASSOCNOTE='{start_aassocnote}']"
    )[0]
    end_element = root.xpath(f"//TITLE[@ATOC='Y' and @AASSOCNOTE='{end_aassocnote}']")[
        0
    ]

    extracted_elements = []
    current_element = start_element
    while current_element is not None:
        extracted_elements.append(
            etree.tostring(current_element, encoding="unicode", with_tail=True)
        )
        if current_element == end_element:
            break
        current_element = current_element.getnext()

    return "".join(extracted_elements)


def extract_audit_report(zip_content: bytes, rcp_no: str) -> str:
    """ZIP 파일에서 감사보고서를 추출합니다."""
    try:
        with zipfile.ZipFile(io.BytesIO(zip_content)) as zf:
            audit_fnames = [
                info.filename
                for info in zf.infolist()
                if rcp_no in info.filename and info.filename.endswith(".xml")
            ]
            if not audit_fnames:
                raise ValueError("감사보고서 파일을 찾을 수 없습니다.")

            xml_data = zf.read(audit_fnames[0])
            parser = etree.XMLParser(recover=True, encoding="utf-8")
            root = etree.fromstring(xml_data, parser)

            return extract_section(root, "D-0-11-2-0", "D-0-11-3-0")

    except (zipfile.BadZipFile, etree.XMLSyntaxError, IndexError) as e:
        logging.error(f"감사보고서 추출 실패: {str(e)}")
        raise


def parse_html_from_xml(xml_data: str) -> etree.Element:
    """XML 데이터를 HTML로 파싱합니다."""
    parser = etree.HTMLParser()
    return etree.fromstring(f"<html><body>{xml_data}</body></html>", parser)


def load_html_with_langchain(html_string: str) -> List[Dict[str, Any]]:
    """HTML 문자열을 LangChain 문서로 로드합니다."""
    with tempfile.NamedTemporaryFile(
        mode="w", encoding="utf-8", suffix=".html", delete=False
    ) as temp_file:
        temp_file.write(html_string)
        temp_file_path = temp_file.name

    try:
        loader = BSHTMLLoader(temp_file_path, open_encoding="utf-8")
        return loader.load()
    finally:
        os.unlink(temp_file_path)


def tsv_string_to_dataframe(tsv_string: str) -> pd.DataFrame:
    """TSV 문자열을 pandas DataFrame으로 변환합니다."""
    return pd.read_csv(io.StringIO(tsv_string), sep="\t", encoding="utf-8")


def process_dataframe(df: pd.DataFrame) -> pd.DataFrame:
    """DataFrame의 데이터 타입을 적절히 변환합니다."""
    for col in df.columns:
        if "금액" in col or "건수" in col:
            df[col] = pd.to_numeric(df[col].replace("-", "0"), errors="coerce")
        elif "일" in col:
            df[col] = pd.to_datetime(df[col], errors="coerce")
    return df


def summarize_report(api_key: str, rcp_no: str) -> pd.DataFrame:
    """보고서를 요약하여 DataFrame으로 반환합니다."""
    try:
        zip_content = fetch_document(api_key, rcp_no)
        logging.info(f"API 응답 크기: {len(zip_content)} 바이트")

        extracted_content = extract_audit_report(zip_content, rcp_no)
        logging.info("XML 섹션 추출 완료")

        root = parse_html_from_xml(extracted_content)
        logging.info("HTML 파싱 완료")

        html_string = etree.tostring(
            root, pretty_print=True, method="html", encoding="unicode"
        )
        docs = load_html_with_langchain(html_string)
        logging.info(f"추출된 문서 수: {len(docs)}")

        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=CHUNK_SIZE, chunk_overlap=CHUNK_OVERLAP
        )
        split_docs = text_splitter.split_documents(docs)

        embeddings = OpenAIEmbeddings()
        vectorstore = FAISS.from_documents(documents=split_docs, embedding=embeddings)
        retriever = vectorstore.as_retriever()

        template = """이 파일에서 '해외채무보증' 또는 '채무보증내역' 제목 아래에 있는 표를 읽어 TSV(Tab-Separated Values) 형식으로 변환해주세요. 다음 지침을 따라주세요:

        1. 표의 두 겹 칼럼 구조를 단일 행 헤더로 변환하세요. 상위 칼럼과 하위 칼럼을 언더스코어(_)로 결합하여 새로운 칼럼 이름을 만드세요.
        예: '채무보증금액'의 하위 칼럼 '제59기말'은 '채무보증금액_제59기말'로 변환

        2. 결과 TSV의 헤더는 다음과 같은 형식이어야 합니다 (실제 칼럼 이름은 원본 표에 따라 다를 수 있음):
        성명    관계    채권자    보증건수    보증기간_시작일    보증기간_종료일    채무보증금액_제59기말    채무보증금액_증가    채무보증금액_감소    채무보증금액_제60기말    채무금액

        3. 데이터 행은 각 칼럼에 해당하는 값을 포함해야 합니다. 값이 없는 경우 빈 칸으로 두지 말고 '-'로 표시하세요.

        4. 숫자 데이터는 쉼표나 기타 구분자 없이 순수한 숫자로 표현하세요.

        5. 날짜는 'YYYY-MM-DD' 형식으로 통일하세요.

        6. TSV 데이터만 반환하세요. 추가 설명이나 주석은 포함하지 마세요.

        7. 결과에 따옴표(''')나 기타 구분자를 포함하지 말고, 순수한 TSV 데이터만 반환하세요.

        #Context:
        {context}

        #Answer:
        """
        prompt = PromptTemplate.from_template(template)
        llm = ChatOpenAI(model=LLM_MODEL, temperature=LLM_TEMPERATURE)
        chain = LLMChain(llm=llm, prompt=prompt)

        context = retriever.get_relevant_documents("")
        tsv_result = chain.run(context=context)

        df = tsv_string_to_dataframe(tsv_result)
        return process_dataframe(df)

    except Exception as e:
        logging.error(f"보고서 요약 중 오류 발생: {str(e)}")
        raise


if __name__ == "__main__":
    api_key = "b0f7f31f54a0f96561f361c405caa204e64c81a1"  # DART API 키
    rcp_no = "20240312000736"  # 문서 번호

    try:
        result_df = summarize_report(api_key, rcp_no)
        print(result_df.head())
        print(result_df.dtypes)
    except Exception as e:
        logging.error(f"프로그램 실행 중 오류 발생: {str(e)}")

2024-07-25 09:23:45,814 - INFO - API 응답 크기: 596351 바이트
2024-07-25 09:23:46,163 - INFO - XML 섹션 추출 완료
2024-07-25 09:23:46,168 - INFO - HTML 파싱 완료
2024-07-25 09:23:46,204 - INFO - 추출된 문서 수: 1
2024-07-25 09:23:47,614 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-07-25 09:23:48,822 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-07-25 09:24:50,507 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


  법인명_채무자    관계         채권자    내용    목적      보증시작일      보증종료일   채무보증한도  \
0     SEA  계열회사       BOA 등  지급보증  운영자금 2023-04-16 2024-12-16  1278000   
1     SEM  계열회사      BBVA 등  지급보증  운영자금 2023-03-28 2024-12-16   715000   
2  SAMCOL  계열회사  Citibank 등  지급보증  운영자금 2023-06-14 2024-12-16   210000   
3    SEDA  계열회사  BRADESCO 등  지급보증  운영자금 2023-10-01 2024-12-16   409000   
4    SECH  계열회사  Citibank 등  지급보증  운영자금 2023-06-14 2024-12-16    62000   

      채무금액 이자율 기초_기말 기초_증감 기말  
0  1278000   -     -     -  -  
1   715000   -     -     -  -  
2   210000   -     -     -  -  
3   329000   -     -     -  -  
4    62000   -     -     -  -  
법인명_채무자            object
관계                 object
채권자                object
내용                 object
목적                 object
보증시작일      datetime64[ns]
보증종료일      datetime64[ns]
채무보증한도             object
채무금액                int64
이자율                object
기초_기말              object
기초_증감              object
기말                 object
dtype: object


In [6]:
result_df

,법인명(채무자),관계,채권자,내용,목적,보증기간_시작일,보증기간_종료일,채무보증한도_기초,채무보증한도_기말,채무금액_기초,채무금액_증감,채무금액_기말,이자율
0,SEA,계열회사,BOA 등,지급보증,운영자금,2023-04-16,2024-12-16,1278000,1278000,0,0,0,NaN
1,SEM,계열회사,BBVA 등,지급보증,운영자금,2023-03-28,2024-12-16,715000,715000,0,0,0,NaN
2,SAMCOL,계열회사,Citibank 등,지급보증,운영자금,2023-06-14,2024-12-16,210000,210000,0,0,0,NaN
3,SEDA,계열회사,BRADESCO 등,지급보증,운영자금,2023-10-01,2024-12-16,409000,329000,0,0,0,NaN
4,SECH,계열회사,Citibank 등,지급보증,운영자금,2023-06-14,2024-12-16,62000,62000,0,0,0,NaN
5,SEPR,계열회사,BBVA 등,지급보증,운영자금,2023-06-01,2024-12-16,150000,150000,0,0,0,NaN
6,SSA,계열회사,SCB 등,지급보증,운영자금,2023-06-14,2024-12-16,318000,318000,0,0,0,NaN
7,SEMAG,계열회사,SocGen 등,지급보증,운영자금,2023-11-09,2024-12-16,110000,96000,0,8065,8065,5.0%
8,SETK,계열회사,BNP 등,지급보증,운영자금,2023-06-14,2024-12-16,777000,917000,239395,81498,320893,48.8%
9,SETK-P,계열회사,BNP 등,지급보증,운영자금,2023-11-09,2024-12-16,130000,70000,25649,-25649,0,NaN
